In [9]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import math
import sys

import skinny
import helpers 
import dpautils

from scipy import stats
from array import array
from operator import xor

np.set_printoptions(threshold=sys.maxsize)

In [10]:
N=15
std=0.5
number_of_experiments=15

TK1_0=[0,1,2,3] #Three times the traces
TK1_1=[4,5,6,7]
TK1_2_unsorted=[0,1,2,3] # Three times the traces
Tk1_3_unsorted=[8,9,10,11]

keys=np.random.randint(2147483647, 9223372036854775807, size=number_of_experiments, dtype=np.int64)  

In [11]:
P=helpers.gen_plaintexts(N)
val=dpautils.compute_intemediate_values(P,int(keys[0]))
interm_values=val[0]
clear_text=val[1]
TK1=val[2].A1
T = dpautils.gen_traces(interm_values, std)

# Attacking TK1_0 (first row of TK1)
Three rows of the internal state depend on the same four Tweakey nibbles. Exploiting this by getting "three traces per trace"

## Simultanous attack

In [12]:
simultanous_success_rate_pr_n=[]
Ns=list(range(1,N,1))
for n in Ns:
    probs=[]
    
    for key in keys:
        TK1_0_guess=[]
        
        # Attack TK1 nibble i (first row of TK1)
        for i in range(4): 
            t=T[:n]
            ct=clear_text[:n]
            nibble_guess=dpautils.simultanous_atk(t,ct,i)
            TK1_0_guess.append(nibble_guess)
            
        #_____SUCCESS/FAIL_________________
        if(np.array_equal(TK1[0:4],TK1_0_guess)):
            probs.append(1)
        else:
            probs.append(0)
    simultanous_success_rate_pr_n.append(np.average(probs))

## Majority Vote

In [13]:
majority_vote_success_rate_pr_n=[]
Ns=list(range(1,N,1))
for n in Ns:
    probs=[]
    
    for key in keys:
        TK1_0_guess=[]
        
        # Attack TK1 nibble i (first row of TK1)
        for i in range(4): 
            t=T[:n]
            ct=clear_text[:n]
            nibble_guess=dpautils.majority_vote_atk(t,ct,i)
            TK1_0_guess.append(nibble_guess)
            
        #_____SUCCESS/FAIL_________________
        if(np.array_equal(TK1[0:4],TK1_0_guess)):
            probs.append(1)
        else:
            probs.append(0)
    majority_vote_success_rate_pr_n.append(np.average(probs))

## unanimous attack

In [14]:
unanimous_success_rate_pr_n=[]
Ns=list(range(1,N,1))
for n in Ns:
    probs=[]
    
    for key in keys:
        TK1_0_guess=[]
        
        # Attack TK1 nibble i (first row of TK1)
        for i in range(4): 
            t=T[:n]
            ct=clear_text[:n]
            nibble_guess=dpautils.unanimous_attack(t,ct,i)
            TK1_0_guess.append(nibble_guess)
            
        #_____SUCCESS/FAIL_________________
        if(np.array_equal(TK1[0:4],TK1_0_guess)):
            probs.append(1)
        else:
            probs.append(0)
    unanimous_success_rate_pr_n.append(np.average(probs))

## Individual attack

In [ ]:
individual_attack_success_rate_pr_n=[]
Ns=list(range(1,N,1))
for n in Ns:
    probs=[]
    
    for key in keys:
        TK1_0_guess=[]
        
        # Attack TK1 nibble i (first row of TK1)
        for i in range(4): 
            t=T[:n]
            ct=clear_text[:n]
            nibble_guess=dpautils.individual_atk(t,ct,i)
            TK1_0_guess.append(nibble_guess)
            
        #_____SUCCESS/FAIL_________________
        if(np.array_equal(TK1[0:4],TK1_0_guess)):
            probs.append(1)
        else:
            probs.append(0)
    individual_attack_success_rate_pr_n.append(np.average(probs))

In [ ]:

outfile="TK1-10-recovery-rate-std-"+str(std)+".png"

helpers.plot_multiple_attacks(simultanous_success_rate_pr_n, majority_vote_success_rate_pr_n, 
                 unanimous_success_rate_pr_n, individual_attack_success_rate_pr_n,
                 "simultanous","majority vote",
                 "unanimous","individual",
                outfile)
    

# Recover second row of TK1

As only a single row of the internal state depend on TK1_1, only individual attack is usable

In [ ]:
individual_attack_success_rate_pr_n=[]
Ns=list(range(1,N,1))
for n in Ns:
    probs=[]
    
    for key in keys:
        TK1_0_guess=[]
        
        # Attack TK1 nibble i (first row of TK1)
        for i in range(5,9,1): 
            t=T[:n]
            ct=clear_text[:n]
            nibble_guess=dpautils.individual_atk(t,ct,i)
            TK1_0_guess.append(nibble_guess)
            
        #_____SUCCESS/FAIL_________________
        if(np.array_equal(TK1[0:4],TK1_0_guess)):
            probs.append(1)
        else:
            probs.append(0)
    individual_attack_success_rate_pr_n.append(np.average(probs))